In [80]:
import pandas as pd
import numpy as np

In [74]:
android_app=pd.read_csv("googleplaystore.csv")
android_app.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [5]:
android_app.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

# Transform the Android Table

In [20]:
android=android_app[['App','Reviews','Rating','Genres','Installs']]
android.head()

,App,Reviews,Rating,Genres,Installs
0,Photo Editor & Candy Camera & Grid & ScrapBook,159,4.1,Art & Design,"10,000+"
1,Coloring book moana,967,3.9,Art & Design;Pretend Play,"500,000+"
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",87510,4.7,Art & Design,"5,000,000+"
3,Sketch - Draw & Paint,215644,4.5,Art & Design,"50,000,000+"
4,Pixel Draw - Number Art Coloring Book,967,4.3,Art & Design;Creativity,"100,000+"


In [21]:
android=android.dropna()

In [23]:
#Sort by the name, and observed that there are duplicated apps being rated multiple times i.e same names appears multiple times with the same or different reviews and ratings
android=android.sort_values("App")
#Trim the spaces of all the columns, covert all columns to a string datatype at this point
android['App']=android['App'].str.strip()
android['Reviews']=android['Reviews'].str.strip()
android['Rating']=android['Rating'].astype(str).str.strip()
android['Genres']=android['Genres'].str.strip()
android['Installs']=android['Installs'].str.strip()
#Change the app name case to a lower case prepare for the further duplicates removal
android['App']=android['App'].str.lower()

In [24]:
#The methodology is if there are duplicated names, we want to take the best reviews and ratings as the result of this app 
#so we created two extra columns and groupby app names to store the max review and ratings
android[['Reviews_max', 'Ratings_max']]=android.groupby('App')['Reviews', 'Rating'].transform('max')
android.head(5)

,App,Reviews,Rating,Genres,Installs,Reviews_max,Ratings_max
8532,+download 4 instagram twitter,40467,4.5,Social,"1,000,000+",40467,4.5
324,- free comics - comic apps,115,3.5,Comics,"10,000+",115,3.5
4541,.r,259,4.5,Tools,"10,000+",259,4.5
4636,/u/app,573,4.7,Communication,"10,000+",573,4.7
5940,058.ba,27,4.4,News & Magazines,100+,27,4.4


In [25]:
#and then we use the max review and ratings as the new review columns and rating columns and prepare for the duplicate removal
new_android=android[['App','Reviews_max','Ratings_max','Genres','Installs']]
new_android.head(5)

,App,Reviews_max,Ratings_max,Genres,Installs
8532,+download 4 instagram twitter,40467,4.5,Social,"1,000,000+"
324,- free comics - comic apps,115,3.5,Comics,"10,000+"
4541,.r,259,4.5,Tools,"10,000+"
4636,/u/app,573,4.7,Communication,"10,000+"
5940,058.ba,27,4.4,News & Magazines,100+


In [26]:
android_table=new_android.drop_duplicates().reset_index(drop=True)
android_table.head(5)

,App,Reviews_max,Ratings_max,Genres,Installs
0,+download 4 instagram twitter,40467,4.5,Social,"1,000,000+"
1,- free comics - comic apps,115,3.5,Comics,"10,000+"
2,.r,259,4.5,Tools,"10,000+"
3,/u/app,573,4.7,Communication,"10,000+"
4,058.ba,27,4.4,News & Magazines,100+


In [27]:
android_table=android_table.rename(columns={"App":"App Name","Reviews_max":"Total_Reviews","Ratings_max":"Ratings"})

In [37]:
android_table.head(5)

,App Name,Total_Reviews,Ratings,Genres,Installs
0,+download 4 instagram twitter,40467,4.5,Social,"1,000,000+"
1,- free comics - comic apps,115,3.5,Comics,"10,000+"
2,.r,259,4.5,Tools,"10,000+"
3,/u/app,573,4.7,Communication,"10,000+"
4,058.ba,27,4.4,News & Magazines,100+


In [38]:
#Convert the numerical fields to a number and found the 3.0M gave us an error
android_table['Ratings']=pd.to_numeric(android_table['Ratings'])
android_table['Total_Reviews']=pd.to_numeric(android_table['Total_Reviews'])

In [45]:
#To fix that, we need to change 3.0M to 3,000,000
#android_table.loc[android_table.Total_Reviews=='3.0M','Total_Reviews']=3000000

In [42]:
#now we are happy
android_table['Total_Reviews']=pd.to_numeric(android_table['Total_Reviews'])

In [44]:
#Check the data type, it looks awesome!
android_table.dtypes

App Name          object
Total_Reviews      int64
Ratings          float64
Genres            object
Installs          object
dtype: object

In [46]:
android_table.head()

,App Name,Total_Reviews,Ratings,Genres,Installs
0,+download 4 instagram twitter,40467,4.5,Social,"1,000,000+"
1,- free comics - comic apps,115,3.5,Comics,"10,000+"
2,.r,259,4.5,Tools,"10,000+"
3,/u/app,573,4.7,Communication,"10,000+"
4,058.ba,27,4.4,News & Magazines,100+


In [65]:
%timeit (android_table["Installs"]).str.replace('+','')

3.04 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [66]:
android_table.head()

,App Name,Total_Reviews,Ratings,Genres,Installs
0,+download 4 instagram twitter,40467,4.5,Social,"1,000,000+"
1,- free comics - comic apps,115,3.5,Comics,"10,000+"
2,.r,259,4.5,Tools,"10,000+"
3,/u/app,573,4.7,Communication,"10,000+"
4,058.ba,27,4.4,News & Magazines,100+


In [67]:
# cleaning the table
final= android_table.sort_values(["Total_Reviews","Ratings"], ascending=False).reset_index(drop=True)

In [68]:
final.head()

,App Name,Total_Reviews,Ratings,Genres,Installs
0,facebook,78158306,4.1,Social,"1,000,000,000+"
1,whatsapp messenger,69119316,4.4,Communication,"1,000,000,000+"
2,instagram,66577446,4.5,Social,"1,000,000,000+"
3,messenger – text and video chat for free,56646578,4.0,Communication,"1,000,000,000+"
4,clash of clans,44893888,4.6,Strategy,"100,000,000+"


In [77]:
final['App Name']=final['App Name'].str.title()
final.head()

,App Name,Total_Reviews,Ratings,Genres,Installs
0,Facebook,78158306,4.1,Social,"1,000,000,000+"
1,Whatsapp Messenger,69119316,4.4,Communication,"1,000,000,000+"
2,Instagram,66577446,4.5,Social,"1,000,000,000+"
3,Messenger – Text And Video Chat For Free,56646578,4.0,Communication,"1,000,000,000+"
4,Clash Of Clans,44893888,4.6,Strategy,"100,000,000+"


In [93]:
from bs4 import BeautifulSoup
import requests


In [94]:
url ='https://www.makeuseof.com/tag/most-popular-android-apps/'
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <link crossorigin="" href="https://static.makeuseof.com" rel="preconnect"/>
  <link href="https://static.h-bid.com" rel="preconnect"/>
  <link href="https://api.monetizer101.com" rel="preconnect"/>
  <link href="https://optimized-by.rubiconproject.com" rel="preconnect"/>
  <link href="https://ads.rubiconproject.com" rel="preconnect"/>
  <link href="https://z-na.amazon-adsystems.com" rel="preconnect"/>
  <link href="https://googleads.g.doubleclick.net" rel="preconnect"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   The 20 Most Popular Android Apps in the Google Play Store
  </title>
  <link href="https://static.makeuseof.com/wp-content/themes/makeuseof2016/style.min.css?v=35" media="all" rel="stylesheet" type="text/css"/>
  <script type="text/javascript">
   var isSponsored = false;
var googletag = googletag || {}; googl

In [106]:
results = soup.find_all('h2')
appnames=[]
for result in results:
    title= result.text
    appnames.append(title)

In [138]:
dic={"Popular_Apps":appnames}
popularApp=pd.DataFrame(dic)

popularApp.head()

,Popular_Apps
0,A Note on Stock Android Apps
1,1. Facebook
2,2. Facebook Messenger
3,3. WhatsApp
4,4. Instagram


In [139]:
popularApp.drop([0]).head(10)

,Popular_Apps
1,1. Facebook
2,2. Facebook Messenger
3,3. WhatsApp
4,4. Instagram
5,5. Skype
6,6. Subway Surfers
7,7. Twitter
8,8. Clean Master: Space Cleaner & Antivirus
9,9. Candy Crush Saga
10,10. Snapchat


In [62]:
#taking mean of ratings.
#Ratings_df = app_overall[['Ratings_x', 'Ratings_y']].mean(axis=1)

In [ ]:
#add a column for best reviews 
#app_overall['Best_Reviews'] = Reviews_df

In [ ]:
#add a column for best rating
#app_overall['Best Ratings'] =Ratings_df

In [ ]:
#app_overall.head()

In [ ]:

#clean_tab = app_overall.drop(['Total_Reviews_x','Total_Reviews_y','Ratings_x','Ratings_y','Genres_y'],axis=1)

In [ ]:
#clean_tab.head()

In [ ]:
#rename the column for genress
#final=clean_tab.rename(columns={"Genres_x":"Genres"})

In [ ]:
#final.head()